In [1]:
env = "production"
env = "staging"


# s3_bucket = "s3://mist-aggregated-stats-{env}/entity_event/entity_event-{env}/".format(env=env)
date_day = "2020-03-1*"
hr = '*'

s3_bucket = "s3://mist-secorapp-staging/sle-coverage-anomaly/sle-coverage-anomaly-staging/".format(env=env)
s3_path = s3_bucket + "dt={day}/hr={hr}/*.seq".format(day=date_day, hr=hr)

print(s3_path)


s3://mist-secorapp-staging/sle-coverage-anomaly/sle-coverage-anomaly-staging/dt=2020-03-1*/hr=*/*.seq


In [2]:
# rdd_capacity = spark.sparkContext.sequenceFile(s3_capacity_path)
# rdd_coverage = spark.sparkContext.sequenceFile(s3_coverage_path)
rdd_coverage = spark.sparkContext.sequenceFile(s3_path)
rdd_coverage.first()

(4109,
 bytearray(b'{"radio_5s":1,"util_nonwifi_std_base":10.686252287455485,"channel":11,"sle_coverage_base":0.9739787605262338,"util_nonwifi_deviation":-0.953568937864141,"rssi_mean":-89.0,"rad_id":"r24","sle_capacity_anomaly_score":-0.001005110795669877,"util_all_mean_base":26.527651431022772,"anomaly_score":2.698591450511145,"dev":"r1","anomaly_type":"weak_signal","band":"24","util_cochannel_std":5.0,"sle_coverage_anomaly":true,"util_cochannel_mean":13.068257183736655,"util_cochannel_std_base":5.0,"events":[],"avg_nclients":1.0,"sle_capacity_error":0.348010216963685,"util_nonwifi_mean_base":14.23794877362038,"util_ap":1.971286614351615,"sle_coverage_error":0.3517354443316347,"util_cochannel_deviation":-3.5514748490356416,"ap":"5c-5b-35-0e-b3-ac","rssi_hist":{"-89":180},"error_rate":0.7976085778719924,"sle_coverage":0.0,"user_seconds_util_high":0,"sle_coverage_base_std":0.13839426885618095,"sle_coverage_anomaly_score":2.698591450511145,"interval":60,"impacted_wlans":["c43d016e-8de6-

In [4]:
import json
# rdd_site = rdd.map(lambda x: json.loads(x[1])).map(lambda x: (x.get("site_id"), x.get("ap_id"))).groupByKey()
# rdd_site = rdd.map(lambda x: json.loads(x[1])).map(lambda x: (x.get("site_id"))).groupBy('site_id')
df_coverage = rdd_coverage.map(lambda x: json.loads(x[1])).toDF() #.map(lambda x: json.loads(x[1])).



/opt/mistsys/ds_incubator/venv/spark-2.4.4-bin-without-hadoop/python/pyspark/sql/session.py:366: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [5]:
# df_capacity
df_coverage.printSchema()

root
 |-- anomaly: string (nullable = true)
 |-- anomaly_score: double (nullable = true)
 |-- anomaly_type: string (nullable = true)
 |-- ap: string (nullable = true)
 |-- avg_nclients: double (nullable = true)
 |-- band: string (nullable = true)
 |-- bandwidth: long (nullable = true)
 |-- channel: long (nullable = true)
 |-- dev: string (nullable = true)
 |-- error_rate: double (nullable = true)
 |-- events: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- impacted_wlans: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- interval: long (nullable = true)
 |-- max_nclients: double (nullable = true)
 |-- org: string (nullable = true)
 |-- power: long (nullable = true)
 |-- rad_id: string (nullable = true)
 |-- radio_5s: long (nullable = true)
 |-- rssi_deviation: double (nullable = true)
 |-- rssi_hist: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)
 |-- rssi_mean: double (nullable = true

In [6]:

# df_capacity.select("org", "site",  "anomaly", "anomaly_score", "anomaly_type", "sle_capacity_anomaly", "util_nonwifi_mean", "util_cochannel_mean").show()

df_coverage_g = df_coverage.select("org", "site", "anomaly_type").groupBy("org", "site", "anomaly_type").count()

In [ ]:
df_coverage_g.orderBy('count', ascending=False).take(10)

# df_capacity_g.sort('count', ascending=False).show() #


In [6]:
# rdd_site.take(1)
rdd_capacity_by_site= rdd_capacity.map(lambda x: json.loads(x[1])).map(lambda x: (x.get("site_id"),1)).countByKey()

rdd_coverage_by_site= rdd_coverage.map(lambda x: json.loads(x[1])).map(lambda x: (x.get("site_id"),1)).countByKey()

In [7]:
site_capacity_sorted = sorted(rdd_capacity_by_site.items(), key=lambda v:v[1], reverse=True) #, key=lambda(k, v): v)
site_coverage_sorted = sorted(rdd_coverage_by_site.items(), key=lambda v:v[1], reverse=True) #, key=lambda(k, v): v)


In [8]:
site_capacity_sorted[:10]


[('5cf9de1a-8034-4472-86f6-9a100ebdb792', 738),
 ('96488ea8-6dee-11e6-a43a-02e208b2d34f', 590),
 ('866808ba-1ff2-4e99-8e17-34ee1d3c70f4', 365),
 ('b3c32004-a0dc-4353-b697-1065fbd4f639', 325),
 ('64f4d304-5015-4777-911f-d71e809397c2', 315),
 ('3cf29ebc-21f1-49e5-9956-25495fe8f41b', 312),
 ('e0e7ada3-59bb-4189-8e1f-ca489812828c', 292),
 ('46247212-c9ab-47f3-ba04-ac38446640a3', 273),
 ('67151fdb-2d69-4e07-a2b9-0652c3d5bec5', 269),
 ('68fdb9ba-7b27-4293-9bf2-ad1c987c9466', 250)]

In [9]:
site_coverage_sorted[:10]

[('5cf9de1a-8034-4472-86f6-9a100ebdb792', 657),
 ('866808ba-1ff2-4e99-8e17-34ee1d3c70f4', 398),
 ('b3c32004-a0dc-4353-b697-1065fbd4f639', 357),
 ('96488ea8-6dee-11e6-a43a-02e208b2d34f', 354),
 ('a799e12a-fc28-4ce7-a563-c2dd6d498e2f', 316),
 ('3cf29ebc-21f1-49e5-9956-25495fe8f41b', 273),
 ('50fa3df4-053f-4779-86d5-f53d74343077', 225),
 ('64f4d304-5015-4777-911f-d71e809397c2', 200),
 ('469fc38e-33da-4562-8b4c-6720c43a53e0', 186),
 ('c14381b7-a3fe-4a85-ac51-11813efded19', 176)]

In [10]:
# rdd_coverage_by_site.sort()

In [15]:

user_org_id = "38a18d4d-1623-4985-86d0-1bb06e5e2a48"  # UPS
user_site_USNYELE = '9aa6ecdb-ddee-41b4-a8d9-872d7962b3c1'  #UPS USNYELE

# user_org_id = '38a18d4d-1623-4985-86d0-1bb06e5e2a48'  # UPS
user_site_USFLLON = 'c23667f3-47e9-44aa-9761-de49d34ed9f9' # USFLLON



In [16]:
rdd_capacity_by_site.get(user_site_USNYELE)

26

In [17]:
rdd_capacity_by_site.get(user_site_USFLLON)

50

In [16]:


# file_path = 's3://mist-secorapp-{env}/oc-stats-analytics/oc-stats-analytics-{env}/dt={date}/*'.format(env=env, date=DATE)

s3_apscan_bucket = 's3://mist-aggregated-stats-{ENV}/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/'.format(ENV=env)


s3_apscan_path = s3_apscan_bucket + "dt={date}/hr={hour}/last_0.25_day/*.csv".format(date=date_day, hour=hr)

print(s3_apscan_path)

# file_path = "s3a://mist-aggregated-stats-production/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/dt=2020-02-26/hr=17/last_0.25_day/part-00000-8b45ee45-d247-4bc9-b4df-b621135297eb-c000.csv"
df_apscan= spark.read.option("header", True).csv(s3_apscan_path)

# df = spark.read.parquet(file_path)
df_apscan.printSchema()


s3://mist-aggregated-stats-production/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/dt=2020-03-03/hr=00/last_0.25_day/*.csv
root
 |-- site: string (nullable = true)
 |-- ap: string (nullable = true)
 |-- ap2: string (nullable = true)
 |-- band: string (nullable = true)
 |-- time_epoch: string (nullable = true)
 |-- rssi: string (nullable = true)



In [17]:
from pyspark.sql.types import DateType

# df_ap.select("time_epoch").show()
df_apscan.withColumn('date', df_apscan['time_epoch'].cast(DateType())).show()


+--------------------+------------+------------+----+----------+-----+----+
|                site|          ap|         ap2|band|time_epoch| rssi|date|
+--------------------+------------+------------+----+----------+-----+----+
|00193f17-4e86-476...|5c5b357eec90|5c5b35aed787|  24|1583197064|-65.0|null|
|00193f17-4e86-476...|5c5b35aed61f|5c5b35aed791|  24|1583197062|-63.0|null|
|00193f17-4e86-476...|5c5b35aed7e6|5c5b35aed7cd|  24|1583191062|-67.0|null|
|00226d33-cd80-4b1...|5c5b35501a7c|        null|  24|1583197014|-61.0|null|
|00272816-8dc2-4f1...|5c5b357f2822|5c5b35ae6ced|  24|1583197063|-67.0|null|
|00272816-8dc2-4f1...|5c5b357f284f|5c5b35ae7152|   5|1583197062|-59.0|null|
|00272816-8dc2-4f1...|5c5b357f32c7|5c5b35ae6d1f|  24|1583196761|-76.0|null|
|00272816-8dc2-4f1...|5c5b357f3308|        null|   5|1583197064|-59.0|null|
|00272816-8dc2-4f1...|5c5b35ae6e55|5c5b35ae6c89|   5|1583197062|-64.0|null|
|00272816-8dc2-4f1...|5c5b35ae6e55|5c5b35ae6d97|   5|1583197062|-69.0|null|
|00272816-8d

In [23]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

# import pyspark
# from pyspark.sql import SparkSession, Row
# from pyspark.sql.types import StringType, IntegerType, ArrayType, FloatType, DataType
# import pyspark.sql.functions as fn
from pyspark.sql.functions import udf, size, avg, count, col,sum, explode
# from operator import itemgetter
# import json, datetime

In [25]:
df_apscan_site = df_apscan.filter(col("site")==("67970e46-4e12-11e6-9188-0242ac110007"))

In [29]:
df_apscan_site.count()


Py4JJavaError: An error occurred while calling o68.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 10 in stage 14.0 failed 1 times, most recent failure: Lost task 10.0 in stage 14.0 (TID 65, localhost, executor driver): org.apache.hadoop.fs.s3a.AWSS3IOException: getFileStatus on s3://mist-aggregated-stats-production/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/dt=2020-03-03/hr=00/last_0.25_day/part-00000-cbf445ab-42fc-42aa-b78f-c79111c394ad-c000.csv: com.amazonaws.services.s3.model.AmazonS3Exception: Bad Request (Service: Amazon S3; Status Code: 400; Error Code: 400 Bad Request; Request ID: D7299B3BAB74A4A9), S3 Extended Request ID: 6SCpTMCwGs4ZxOZ/Y1xG9ODJTwAAcK/+ZWoSILeUqQZhQI/s1ynWlgeMZCksbVXtNwWB8BvH0e0=: Bad Request (Service: Amazon S3; Status Code: 400; Error Code: 400 Bad Request; Request ID: D7299B3BAB74A4A9)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:178)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateExcept

In [28]:
df_apscan_site.show()

+----+---+---+----+----------+----+
|site| ap|ap2|band|time_epoch|rssi|
+----+---+---+----+----------+----+
+----+---+---+----+----------+----+



In [ ]:
from graphframes import *